# Cycling Part 2: Country Comparisons 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='pandas')
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dublin= pd.read_csv("Cycling.csv")

# Dublin

In [ ]:
dublin.head()

In [ ]:
dublin = dublin.drop('bikes_in_use', axis=1)

In [ ]:
print("Minimum date:", dublin['date'].min())
print("Maximum date:", dublin['date'].max())

In [ ]:
dublin['datetime'] = pd.to_datetime(dublin['date'].astype(str) + ' ' + dublin['time'].astype(str))

In [ ]:
dublin.isnull().sum()

In [ ]:
dublin.describe()

In [ ]:
dublin.index = pd.to_datetime(dublin.index)

In [ ]:
# Features and target variable
X = dublin[['capacity', 'num_docks_available']]
y = dublin['num_bikes_available']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Split the data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### LinearRegression model 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

print('Linear Regression Mean Squared Error:', mean_squared_error(y_test, y_pred_lr))
print('Linear Regression R-squared:', r2_score(y_test, y_pred_lr))

### Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print('Random Forest Mean Squared Error:', mean_squared_error(y_test, y_pred_rf))
print('Random Forest R-squared:', r2_score(y_test, y_pred_rf))

The Random Forest model has an MSC level lower 0.98, making it the best model for training the available bicycle data in Dublin

### Forecast using Prophet model

In [ ]:
from prophet import Prophet

In [ ]:
station_data = dublin[dublin['station_id'] == 92][['datetime', 'num_bikes_available']]

station_data = station_data.set_index('datetime').resample('D').mean().reset_index()

df_prophet = station_data.rename(columns={'datetime': 'ds', 'num_bikes_available': 'y'})

In [ ]:
model_prophet = Prophet()
model_prophet.fit(df_prophet)

future = model_prophet.make_future_dataframe(periods=7) 


forecast = model_prophet.predict(future)

forecast_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)
print(forecast_df)

In [ ]:
fig = model_prophet.plot(forecast)
plt.title('Bike Availability Forecast')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(forecast['ds'], forecast['yhat'], label='Prediction', color='green')  
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='green', alpha=0.2)  
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Predictions with Confidence Intervals')
plt.legend()
plt.show()

# Mexico City and New York

In [ ]:
Cycling = pd.read_csv("Cycling2.csv")

#### Exploratory Data Analysis (EDA)

In [ ]:
Cycling.head()

In [ ]:
Cycling = Cycling.drop(Cycling.columns[0], axis=1)

In [ ]:
Cycling.dtypes

In [ ]:
Cycling.isnull().sum()

#### Analysis of the same time period 

In [ ]:
Cycling['time_started'] = pd.to_datetime(Cycling['time_started'], format='%H:%M:%S').dt.time
Cycling['time_end'] = pd.to_datetime(Cycling['time_end'], format='%H:%M:%S').dt.time

In [ ]:
# datetime_started
Cycling['datetime_started'] = pd.to_datetime(Cycling['date_started'].astype(str) + ' ' + Cycling['time_started'].astype(str))

# datetime_end
Cycling['datetime_end'] = pd.to_datetime(Cycling['date_end'].astype(str) + ' ' + Cycling['time_end'].astype(str))

In [ ]:
ny_data = Cycling[Cycling['Country'] == 'New_York']
mexico_data = Cycling[Cycling['Country'] == 'Mexico']

In [ ]:
# Dates per City
ny_dates = ny_data['date_started'].unique()
mexico_dates = mexico_data['date_started'].unique()

In [ ]:
#Period
print("Nueva York:", ny_data['date_started'].min(), "to", ny_data['date_started'].max())
print("Mexico:", mexico_data['date_started'].min(), "to", mexico_data['date_started'].max())

In [ ]:
#Data filtered

In [ ]:
Cycling['date_started'] = pd.to_datetime(Cycling['date_started'])
start_date = pd.to_datetime('2024-05-31')
end_date = pd.to_datetime('2024-06-17')

In [ ]:
cycling1 = filtered_data = Cycling[(Cycling['date_started'] >= start_date) & (Cycling['date_started'] <= end_date)]

In [ ]:
cycling = cycling1.copy()

In [ ]:
cycling.shape

### Range

In [ ]:
trips_more_60min = (cycling['duration_minutes'] >60 ).sum()
print(f"Number of trips lasting more than 1 hour: {trips_more_60min}")

In [ ]:
bins = [0, 5, 10, 15, 20, 30, 40, 50, 60,float('inf')]
labels = ['0-5', '5-10', '10-15', '15-20', '20-30', '30-40', '40-50', '50-60', '60+']

# Convert 'duration_minutes' to numeric, if not already
cycling['duration_minutes'] = pd.to_numeric(cycling['duration_minutes'], errors='coerce')

# Create the 'Range' column based on the defined intervals
cycling['Range'] = pd.cut(cycling['duration_minutes'], bins=bins, labels=labels, right=False)

In [ ]:
cycling_range = cycling[['date_started', 'Country', 'Range']]

In [ ]:
# Filter data by country
cycling_ny = cycling_range[cycling_range['Country'] == 'New_York']
cycling_mexico = cycling_range[cycling_range['Country'] == 'Mexico']

# Calculate the distribution of duration ranges for each country
ny_distribution = cycling_ny['Range'].value_counts().sort_index()
mexico_distribution = cycling_mexico['Range'].value_counts().sort_index()

# Define colors for each range
colors = {
    '0-5': '#008000',
    '5-10': '#BFBF00',
    '10-15': '#556B2F',
    '15-20': '#66CDAA',
    '20-30': '#008B8B',
    '30-40': '#c2f0c2',
    '40-50': '#ff6666',
    '50-60': '#5F9EA0',
    '60+': '#ff9999'
}

# Create pie charts with improved readability
fig, axes = plt.subplots(1, 2, figsize=(18, 8))  # Increase figure size for better visibility

# Pie chart for New York
sizes = ny_distribution
labels = ny_distribution.index
ny_colors = [colors[label] for label in labels]  # Get colors for New York ranges

def autopct_format(pct):
    return f'{pct:.1f}%' if pct > 2 else ''  # Hide small percentages

axes[0].pie(sizes, 
            labels=labels, 
            autopct=autopct_format, 
            colors=ny_colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1},  # White border around slices
            textprops={'color': 'white'},  # White text color
            labeldistance=1.2,  # Move labels away from the center
            explode=[0.1 if x < 10 else 0 for x in sizes])  # Explode small slices
axes[0].set_title('Duration Range Distribution - New York', color='white')

# Pie chart for Mexico
sizes = mexico_distribution
labels = mexico_distribution.index
mexico_colors = [colors[label] for label in labels]  # Get colors for Mexico ranges

axes[1].pie(sizes, 
            labels=labels, 
            autopct=autopct_format, 
            colors=mexico_colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1},  # White border around slices
            textprops={'color': 'white'},  # White text color
            labeldistance=1.2,  # Move labels away from the center
            explode=[0.1 if x < 10 else 0 for x in sizes])  # Explode small slices
axes[1].set_title('Duration Range Distribution - Mexico', color='white')

# Adjust layout and set background color
plt.tight_layout()
fig.patch.set_facecolor('black')  # Set background color of the figure to black
axes[0].set_facecolor('black')  # Set background color of the subplot
axes[1].set_facecolor('black')  # Set background color of the subplot

# Show the pie charts
plt.show()

In [ ]:
population_ny = 8_000_000  # New York
population_mx = 9_209_944  # Mexico City

### Duration_minutes

In [ ]:
cycling1 = cycling.drop(['Range','date_end', 'time_started', 'time_end'], axis=1)

In [ ]:
print(cycling['duration_minutes'].describe())

In [ ]:
cycling1.head()

In [ ]:
# Extract hour from datetime_started and create a new column 'hour'
cycling1['hour_started'] = cycling1['datetime_started'].dt.hour
cycling1['hour_end'] = cycling1['datetime_end'].dt.hour

# To handle cases where 'hour_end' may be different than 'hour_started', we will consider the whole range
# Expand the DataFrame to cover each hour between 'hour_started' and 'hour_end'
expanded_data = []

for _, row in cycling1.iterrows():
    start_hour = row['hour_started']
    end_hour = row['hour_end']
    for hour in range(start_hour, end_hour + 1):
        expanded_data.append({
            'date_started': row['date_started'],
            'hour': hour,
            'duration_minutes': row['duration_minutes'],
            'bikes_in_use': row['bikes_in_use'],
            'Country': row['Country']
        })

expanded_df = pd.DataFrame(expanded_data)

# Aggregate data by date and hour
df = expanded_df.groupby(['date_started', 'hour', 'Country']).agg({
    'duration_minutes': 'sum',  # Sum duration minutes for each hour
    'bikes_in_use': 'sum'       # Sum bikes in use for each hour
}).reset_index()

In [ ]:
df

#### Missing value

In [ ]:
# Define the range of dates and hours
start_date = pd.to_datetime(df['date_started'].min())
end_date = pd.to_datetime(df['date_started'].max())
all_hours = pd.date_range(start=start_date, end=end_date, freq='H')

# Create a DataFrame with all possible hours
all_hours_df = pd.DataFrame({
    'date_started': all_hours.date,
    'hour': all_hours.hour
})

In [ ]:
# Filter data for New York and Mexico
ny_data = df[df['Country'] == 'New_York']
mx_data = df[df['Country'] == 'Mexico']

# Combine date and hour into a single column for both the complete hours DataFrame and the filtered data
ny_data['datetime'] = pd.to_datetime(ny_data['date_started'].astype(str) + ' ' + ny_data['hour'].astype(str) + ':00:00')
mx_data['datetime'] = pd.to_datetime(mx_data['date_started'].astype(str) + ' ' + mx_data['hour'].astype(str) + ':00:00')

all_hours_df['datetime'] = pd.to_datetime(all_hours_df['date_started'].astype(str) + ' ' + all_hours_df['hour'].astype(str) + ':00:00')

# Check coverage
ny_hours_covered = all_hours_df['datetime'].isin(ny_data['datetime'])
mx_hours_covered = all_hours_df['datetime'].isin(mx_data['datetime'])

In [ ]:
# Find missing hours
missing_ny_hours = all_hours_df[~ny_hours_covered]
missing_mx_hours = all_hours_df[~mx_hours_covered]

# Print the results
print("Missing hours in New York:")
print(missing_ny_hours)

print("Missing hours in Mexico:")
print(missing_mx_hours)

#### Normalize the relevant columns

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Add a column for normalized duration_minutes and bikes_in_use
df['duration_minutes_normalized'] = df.apply(
    lambda row: row['duration_minutes'] / population_ny if row['Country'] == 'New_York' 
                else row['duration_minutes'] / population_mx, axis=1
)

df['bikes_in_use_normalized'] = df.apply(
    lambda row: row['bikes_in_use'] / population_ny if row['Country'] == 'New_York'
                else row['bikes_in_use'] / population_mx, axis=1
)

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
df[['duration_minutes_normalized', 'bikes_in_use_normalized']] = scaler.fit_transform(
    df[['duration_minutes_normalized', 'bikes_in_use_normalized']]
)

# Display the first few rows of the dataframe to verify
print(df.head())

#### Standardize the relevant columns

In [ ]:
scaler = StandardScaler()

df[['duration_minutes', 'bikes_in_use']] = scaler.fit_transform(
    df[['duration_minutes', 'bikes_in_use']]
)

In [ ]:
# Normalize the relevant columns
df[['duration_minutes', 'bikes_in_use']] = scaler.fit_transform(
    df[['duration_minutes', 'bikes_in_use']]
)

In [ ]:
# Standardize duration_minutes and bikes_in_use by country
def standardize_by_country(group):
    group[['duration_minutes_standardized', 'bikes_in_use_standardized']] = scaler.fit_transform(
        group[['duration_minutes', 'bikes_in_use']]
    )
    return group

# Apply standardization
df_standardized = df.groupby('Country').apply(standardize_by_country).reset_index(drop=True)

In [ ]:
def adjust_for_population(row):
    if row['Country'] == 'New_York':
        population = population_ny
    elif row['Country'] == 'Mexico':
        population = population_mx
    else:
        population = 1  # Default to 1 to avoid division by zero
    
    # Scale standardized values by population
    row['duration_minutes_adjusted'] = row['duration_minutes_standardized'] * (population / population_ny)
    row['bikes_in_use_adjusted'] = row['bikes_in_use_standardized'] * (population / population_ny)
    return row

# Apply population adjustment
df_final = df_standardized.apply(adjust_for_population, axis=1)

In [ ]:
cycling_hour = df_final.drop(['duration_minutes','bikes_in_use'], axis=1)

In [ ]:
cycling_hour.head()

## Analysis

#### Standardized Duration Minutes Distribution

In [ ]:
ny_data = cycling_hour[cycling_hour['Country'] == 'New_York']
mx_data = cycling_hour[cycling_hour['Country'] == 'Mexico']

In [ ]:

plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.hist(ny_data['duration_minutes_standardized'], bins=30, color='blue', alpha=0.7, label='New York')
plt.hist(mx_data['duration_minutes_standardized'], bins=30, color='orange', alpha=0.7, label='Mexico')
plt.title('Standardized Duration Minutes Distribution')
plt.xlabel('Standardized Duration Minutes')
plt.ylabel('Frequency')
plt.legend()

### Standardized Bikes in Use

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 2)
plt.hist(ny_data['bikes_in_use_standardized'], bins=30, color='blue', alpha=0.7, label='New York')
plt.hist(mx_data['bikes_in_use_standardized'], bins=30, color='orange', alpha=0.7, label='Mexico')
plt.title('Standardized Bikes in Use Distribution')
plt.xlabel('Standardized Bikes in Use')
plt.ylabel('Frequency')
plt.legend()

### Adjusted Duration Minutes

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 3)
plt.hist(ny_data['duration_minutes_adjusted'], bins=30, color='blue', alpha=0.7, label='New York')
plt.hist(mx_data['duration_minutes_adjusted'], bins=30, color='orange', alpha=0.7, label='Mexico')
plt.title('Adjusted Duration Minutes Distribution')
plt.xlabel('Adjusted Duration Minutes')
plt.ylabel('Frequency')
plt.legend()

#### Histogram for 'bikes_in_use_adjusted'

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 4)
plt.hist(ny_data['bikes_in_use_adjusted'], bins=30, color='blue', alpha=0.7, label='New York')
plt.hist(mx_data['bikes_in_use_adjusted'], bins=30, color='orange', alpha=0.7, label='Mexico')
plt.title('Adjusted Bikes in Use Distribution')
plt.xlabel('Adjusted Bikes in Use')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()

plt.show()

### Standardized Duration Minutes by Hour of the Day

In [ ]:
combined_df = pd.concat([
    ny_data.assign(Country='New_York'),
    mx_data.assign(Country='Mexico')
])

# Set the figure size
plt.figure(figsize=(12, 6))

# Line plot for 'duration_minutes_standardized'
plt.subplot(2, 2, 1)
sns.lineplot(data=combined_df, x='hour', y='duration_minutes_standardized', hue='Country', marker='o')
plt.title('Standardized Duration Minutes by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Standardized Duration Minutes')

## Standardized Bikes in Use by Hour of the Day

In [ ]:
plt.subplot(2, 2, 2)
sns.lineplot(data=combined_df, x='hour', y='bikes_in_use_standardized', hue='Country', marker='o')
plt.title('Standardized Bikes in Use by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Standardized Bikes in Use')

### Adjusted Duration Minutes by Hour of the Day

In [ ]:
plt.subplot(2, 2, 3)
sns.lineplot(data=combined_df, x='hour', y='duration_minutes_adjusted', hue='Country', marker='o')
plt.title('Adjusted Duration Minutes by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Adjusted Duration Minutes')

### Adjusted Bikes in Use by Hour of the Day

In [ ]:
# Line plot for 'bikes_in_use_adjusted'
plt.subplot(2, 2, 4)
sns.lineplot(data=combined_df, x='hour', y='bikes_in_use_adjusted', hue='Country', marker='o')
plt.title('Adjusted Bikes in Use by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Adjusted Bikes in Use')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

### Boxplot of Normalized Trip Duration by Country

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 6))

# Boxplot for 'duration_minutes_normalized' by country
sns.boxplot(data=df, x='Country', y='duration_minutes_normalized')
plt.title('Boxplot of Normalized Trip Duration by Country')
plt.xlabel('Country')
plt.ylabel('Normalized Trip Duration in Minutes')
plt.show()

### Boxplot of Normalized Bikes in Use by Country

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 6))

# Boxplot for 'bikes_in_use_normalized' by country
sns.boxplot(data=df, x='Country', y='bikes_in_use_normalized')
plt.title('Boxplot of Normalized Bikes in Use by Country')
plt.xlabel('Country')
plt.ylabel('Normalized Number of Bikes in Use')
plt.show()

#### Usage Over Time

In [ ]:
cycling_hour.rename(columns={'date_started': 'date'}, inplace=True)

In [ ]:
cycling_hour.head()

In [ ]:
# Aggregate total normalized duration per day and country
daily_usage = cycling_hour.groupby(['date', 'Country']).agg(
    duration_minutes_normalized=('duration_minutes_normalized', 'sum')  
).reset_index()

# Set the figure size
plt.figure(figsize=(14, 7))

# Plot the line plot
sns.lineplot(data=daily_usage, x='date', y='duration_minutes_normalized', hue='Country', marker='o')

# Set the title and labels
plt.title('Trend of Normalized Bicycle Usage Over Time by Country')
plt.xlabel('Date')
plt.ylabel('Total Normalized Duration in Minutes')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)  

# Show grid
plt.grid(True) 

# Adjust layout to prevent clipping
plt.tight_layout()  

# Add legend with title
plt.legend(title='Country')  

# Display the plot
plt.show()

In [ ]:
df = cycling_hour

## MODELS

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

In [ ]:
# Date and hour to a datetime index
df['datetime'] = pd.to_datetime(df['date']) + pd.to_timedelta(df['hour'], unit='h')
df.set_index('datetime', inplace=True)
df = df.loc[~df.index.duplicated(keep='first')]

# Daily frequency 
df = df.asfreq('H')  

## Mexico

In [ ]:
# Date and hour to a datetime index
df['datetime'] = pd.to_datetime(df['date']) + pd.to_timedelta(df['hour'], unit='h')
df.set_index('datetime', inplace=True)
df = df.loc[~df.index.duplicated(keep='first')]

# Daily frequency 
df = df.asfreq('H')  

# Filter data for Mexico
df_mexico = df[df['Country'] == 'Mexico']

In [ ]:
start_date = pd.to_datetime('2024-05-31')
test_start_date = pd.to_datetime('2024-06-10')

In [ ]:
df_mexico.index = pd.to_datetime(df_mexico.index)

In [ ]:
train_mexico = df_mexico.loc[df_mexico.index < test_start_date, 'bikes_in_use_normalized']
test_mexico= df_mexico.loc[(df_mexico.index >= test_start_date) & (df_mexico.index <= '2024-06-17'), 'bikes_in_use_normalized']

### ARIMA model

In [ ]:
# Defining ranges for p, d, q parameters
p = range(0, 5)
d = range(0, 2)
q = range(0, 5)

# All combinations of p, d, q
pdq = list(itertools.product(p, d, q))

best_score, best_cfg = float("inf"), None

for order in pdq:
    try:
        # Fit the ARIMA model
        model_fit = fit_arima(train_mexico, order=order)
        # Forecast
        forecast_mexico = model_fit.forecast(steps=len(test_mexico))
        # Calculate MSE
        mse_mexico = mean_squared_error(test_mexico, forecast_mexico)
        print(f"ARIMA{order} MSE: {mse_mexico}")
        
        if mse_mexico < best_score:
            best_score, best_cfg = mse_mexico, order
    except:
        continue

print(f'Best ARIMA{best_cfg} MSE: {best_score}')

In [ ]:
def fit_arima(train_data, order=(2,0,3)):
    model = ARIMA(train_data, order=order)
    model_fit = model.fit()
    return model_fit

# Fitting the model
model_fit_mexico = fit_arima(train_mexico)

In [ ]:
forecast_mexico = model_fit_mexico.forecast(steps=len(test_mexico))

In [ ]:
mse_mexico = mean_squared_error(test_mexico, forecast_mexico)
print(f"Mean Squared Error for Mexico (ARIMA): {mse_mexico}")

In [ ]:
# Plot results
plt.figure(figsize=(14, 7))
plt.plot(test_mexico.index, test_mexico, label='Actual Mexico')
plt.plot(test_mexico.index, forecast_mexico, label='Forecast Mexico', color='red')
plt.title('ARIMA Forecast vs Actual for Mexico')
plt.xlabel('Date')
plt.ylabel('Bikes in Use')
plt.legend()
plt.show()

In [ ]:
# ARIMA Residuals
residuals_arima = train_mexico - model_fit_mexico.fittedvalues
plt.figure(figsize=(12, 6))
plt.plot(residuals_arima)
plt.title('ARIMA Residuals')
plt.show()

### SARIMAX model

In [ ]:
import random

In [ ]:
def fit_sarima(train_data, order, seasonal_order):
    try:
        model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        return model_fit
    except Exception as e:
        print(f"Error fitting SARIMA{order} x {seasonal_order}: {e}")
        return None

# Definir rangos para parámetros
p = range(0, 3)
d = range(0, 2)
q = range(0, 3)
P = range(0, 3)
D = range(0, 2)
Q = range(0, 3)
s = [24]  # Período estacional

# Generar combinaciones de parámetros
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, s))

# Crear lista de todas las combinaciones
all_combinations = list(itertools.product(pdq, seasonal_pdq))

# Seleccionar una muestra aleatoria si hay más de 20 combinaciones
if len(all_combinations) > 20:
    all_combinations = random.sample(all_combinations, 20)

best_mse = float("inf")
best_cfg = None
best_model = None

for count, (order, seasonal_order) in enumerate(all_combinations, start=1):
    print(f"Iteration {count} of {len(all_combinations)}")
    
    # Verificar la longitud de la temporada
    print(f"Trying SARIMAX{order} x {seasonal_order}")
    
    model_fit = fit_sarima(train_mexico, order=order, seasonal_order=seasonal_order)
    
    if model_fit:
        try:
            # Pronóstico
            forecast_mexico = model_fit.get_forecast(steps=len(test_mexico)).predicted_mean
            
            # Verificar longitud del pronóstico
            if len(forecast_mexico) != len(test_mexico):
                print(f"Warning: Forecast length ({len(forecast_mexico)}) does not match test length ({len(test_mexico)})")
                continue

            # Calcular MSE
            mse_mexico = mean_squared_error(test_mexico, forecast_mexico)
            print(f"SARIMAX{order} x {seasonal_order} MSE: {mse_mexico}")

            # Actualizar el mejor modelo si corresponde
            if mse_mexico < best_mse:
                best_mse = mse_mexico
                best_cfg = (order, seasonal_order)
                best_model = model_fit

        except Exception as e:
            print(f"Error during forecast or MSE calculation for SARIMAX{order} x {seasonal_order}: {e}")

print(f'Best SARIMAX{best_cfg} with MSE: {best_mse}')

In [ ]:
# Function to fit 
def fit_sarimax(train_data, order=(1,0,0), seasonal_order=(2,1,1,24)):
    model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    return model_fit

# Fitting model
model_fit_mexico_sarimax = fit_sarimax(train_mexico)

In [ ]:
forecast_mexico_sarimax = model_fit_mexico_sarimax.get_forecast(steps=len(test_mexico)).predicted_mean

mse_mexico_sarimax = mean_squared_error(test_mexico, forecast_mexico_sarimax)

print(f"Mean Squared Error for Mexico (SARIMAX): {mse_mexico_sarimax}")

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(test_mexico.index, test_mexico, label='Actual Mexico')
plt.plot(test_mexico.index, forecast_mexico_sarimax, label='Forecast Mexico (SARIMAX)', color='red')
plt.title('SARIMAX Forecast vs Actual for Mexico')
plt.xlabel('Date')
plt.ylabel('Bikes in Use')
plt.legend()
plt.show()

In [ ]:
# SARIMAX Residuals
residuals_sarimax = train_mexico - model_fit_mexico_sarimax.fittedvalues
plt.figure(figsize=(12, 6))
plt.plot(residuals_sarimax)
plt.title('SARIMAX Residuals')
plt.show()

## New York

In [ ]:
df_ny = df[df['Country'] == 'New_York']

In [ ]:
train_ny = df_ny.loc[df_ny.index < '2024-06-10']['bikes_in_use_normalized']
test_ny = df_ny.loc[df_ny.index >= '2024-06-10']['bikes_in_use_normalized']

In [ ]:
df_ny.index = pd.to_datetime(df_ny.index)

# Define las fechas de corte para el entrenamiento y la prueba
start_date = pd.to_datetime('2024-05-31')
test_start_date = pd.to_datetime('2024-06-10')

# Filtra los datos para los conjuntos de entrenamiento y prueba
train_ny = df_ny.loc[df_ny.index < test_start_date, 'bikes_in_use_normalized']
test_ny = df_ny.loc[(df_ny.index >= test_start_date) & (df_ny.index <= '2024-06-17'), 'bikes_in_use_normalized']

## ARIMA MODEL

### High residues 

In [ ]:
p = range(0, 5)
d = range(0, 2)
q = range(0, 5)

pdq = list(itertools.product(p, d, q))

best_score, best_cfg = float("inf"), None

for order in pdq:
    try:
        model_fit = fit_arima(train_ny, order=order)
        residuals = model_fit.resid
        forecast_ny = model_fit.forecast(steps=len(test_ny))
        mse_ny = mean_squared_error(test_ny, forecast_ny)
        print(f"ARIMA{order} MSE: {mse_ny}")
        
        if mse_ny < best_score:
            best_score, best_cfg = mse_ny, order
    except:
        continue

print(f'Best ARIMA{best_cfg} MSE: {best_score}')

In [ ]:
def fit_arima(train_data, order=(0,0,3)):
    model = ARIMA(train_data, order=order)
    model_fit = model.fit()
    return model_fit

# Fitting the model
model_fit_ny = fit_arima(train_ny)

In [ ]:
forecast_ny = model_fit_ny.forecast(steps=len(test_ny))

In [ ]:
mse_ny = mean_squared_error(test_ny, forecast_ny)
print(f"Mean Squared Error for New York (ARIMA): {mse_ny}")

In [ ]:
# ARIMA Residuals
residuals_arima = train_ny - model_fit_ny.fittedvalues
plt.figure(figsize=(12, 6))
plt.plot(residuals_arima)
plt.title('ARIMA Residuals')
plt.show()

In [ ]:
# Fit the best ARIMA model
best_order = (0, 0, 3)
model_fit = fit_arima(train_ny, order=best_order)

residuals = model_fit.resid
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(residuals)
plt.title('Residuals of the Best ARIMA Model')
plt.subplot(2, 1, 2)
plt.hist(residuals, bins=50)
plt.title('Histogram of Residuals')
plt.tight_layout()
plt.show()

### SARIMAX MODEL

In [ ]:
def fit_sarima(train_data, order, seasonal_order):
    try:
        model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        return model_fit
    except Exception as e:
        print(f"Error fitting SARIMA{order} x {seasonal_order}: {e}")
        return None

# Definir rangos para parámetros
p = range(0, 3)
d = range(0, 2)
q = range(0, 3)
P = range(0, 3)
D = range(0, 2)
Q = range(0, 3)
s = [24]  # Período estacional

# Generar combinaciones de parámetros
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, s))

# Crear lista de todas las combinaciones
all_combinations = list(itertools.product(pdq, seasonal_pdq))

# Seleccionar una muestra aleatoria si hay más de 20 combinaciones
if len(all_combinations) > 20:
    all_combinations = random.sample(all_combinations, 20)

best_mse = float("inf")
best_cfg = None
best_model = None

for count, (order, seasonal_order) in enumerate(all_combinations, start=1):
    print(f"Iteration {count} of {len(all_combinations)}")
    
    # Verificar la longitud de la temporada
    print(f"Trying SARIMAX{order} x {seasonal_order}")
    
    model_fit = fit_sarima(train_ny, order=order, seasonal_order=seasonal_order)
    
    if model_fit:
        try:
            # Pronóstico
            forecast_ny = model_fit.get_forecast(steps=len(test_ny)).predicted_mean
            
            # Verificar longitud del pronóstico
            if len(forecast_ny) != len(test_ny):
                print(f"Warning: Forecast length ({len(forecast_ny)}) does not match test length ({len(test_ny)})")
                continue

            # Calcular MSE
            mse_ny = mean_squared_error(test_ny, forecast_ny)
            print(f"SARIMAX{order} x {seasonal_order} MSE: {mse_ny}")

            # Actualizar el mejor modelo si corresponde
            if mse_ny < best_mse:
                best_mse = mse_ny
                best_cfg = (order, seasonal_order)
                best_model = model_fit

        except Exception as e:
            print(f"Error during forecast or MSE calculation for SARIMAX{order} x {seasonal_order}: {e}")

print(f'Best SARIMAX{best_cfg} with MSE: {best_mse}')

#### Forecast

In [ ]:
#Mexico
#ARIMA model
model_fit_mexico = fit_arima(train_mexico)

# Forecast steps for 7 days (168 hours)
forecast_steps = 7 * 24  

forecast_index = pd.date_range(start=train_mexico.index[-1] + pd.Timedelta(hours=1), periods=forecast_steps, freq='H')

forecast_mexico = model_fit_mexico.forecast(steps=forecast_steps)

forecast_df = pd.DataFrame({
    'Datetime': forecast_index,
    'Forecast': forecast_mexico
})

#New York 
# Fit the ARIMA model
model_fit_mexico = fit_arima(train_mexico)

# Define the forecast steps for 7 days (168 hours)
forecast_steps = 7 * 24  # 7 days * 24 hours/day = 168 hours

# Create the date range index for the forecast
forecast_index = pd.date_range(start=train_mexico.index[-1] + pd.Timedelta(hours=1), periods=forecast_steps, freq='H')

# Make the forecast with ARIMA
forecast_mexico = model_fit_mexico.forecast(steps=forecast_steps)

# Create a DataFrame for the ARIMA forecast results
forecast_df = pd.DataFrame({
    'Datetime': forecast_index,
    'Forecast': forecast_mexico
})

# Display the ARIMA forecast results
print(forecast_df)

# Fit the SARIMAX model
model_fit_mexico_sarimax = fit_sarimax(train_mexico)

# Make the forecast with SARIMAX
forecast_mexico_sarimax = model_fit_mexico_sarimax.get_forecast(steps=forecast_steps).predicted_mean

# Create a DataFrame for the SARIMAX forecast results
forecast_df_sarimax = pd.DataFrame({
    'Datetime': forecast_index,
    'Forecast_SARIMAX': forecast_mexico_sarimax
})

# Display the SARIMAX forecast results
print(forecast_df_sarimax)

### Forecast

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(cycling_hour.index, cycling_hour['bikes_in_use_normalized'], label='Actual')
plt.plot(forecast_df.index, forecast_df['forecast'], label='Forecast', color='red')
plt.title('Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('bikes_in_use_normalized')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(cycling_hour.index, cycling_hour['duration_minutes'], label='Actual')
plt.plot(forecast_7_days['ds'], forecast_7_days['yhat'], label='Forecast', color='red')
plt.title('Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Duration in Minutes')
plt.legend()
plt.show()

### Forecast by Country

In [ ]:
# Dataframe
future_dates = pd.date_range(start=pd.Timestamp.today(), periods=7, freq='D')
future_df = pd.DataFrame({
    'day_of_week': future_dates.dayofweek,
    'hour_started': 12,
    'month': future_dates.month
})

# Predict durations for the next 7 days by country
for country, model in models.items():
    future_predictions = model.predict(future_df)
    predictions[country] = future_predictions

    
for country, preds in predictions.items():
    print(f'Predicted Durations for the Next 7 Days in {country}:')
    for date, pred in zip(future_dates, preds):
        print(f'{date.date()}: {pred:.2f} minutes')
    print()

In [ ]:
plt.figure(figsize=(14, 8))

for country, preds in predictions.items():
    plt.plot(future_dates, preds, marker='o', label=country)

plt.title('Predicted Bicycle Usage Duration for the Next 7 Days')
plt.xlabel('Date')
plt.ylabel('Predicted Duration (Minutes)')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()